In [3]:
!pip install tensorflow
!pip install matplotlib

In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

In [5]:
from google.colab import drive
import zipfile
import os

# Mount Google Drive
drive.mount('/content/drive')

# Path ke file zip di Google Drive
# zip_file_path = '/content/drive/MyDrive/cnnmakananbayi/dataset.zip'
extract_folder = '/content/drive/MyDrive/cnnmakananbayi/dataset'

# # Ekstrak file zip
# with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
#     zip_ref.extractall(extract_folder)

# Cek isi folder
os.listdir(extract_folder)

Mounted at /content/drive


['train', 'test']

In [6]:
# from PIL import Image
# import os
# Tentukan path untuk folder train dan test
train_dir = os.path.join(extract_folder, 'train')
test_dir = os.path.join(extract_folder, 'test')

# def check_and_remove_corrupted_images(folder_path):
#     for subdir, _, files in os.walk(folder_path):
#         for file in files:
#             file_path = os.path.join(subdir, file)
#             try:
#                 img = Image.open(file_path)
#                 img.verify()  # Verifikasi gambar
#             except (IOError, SyntaxError) as e:
#                 print(f'Removing corrupted image: {file_path}')
#                 os.remove(file_path)

# # Periksa dan hapus gambar yang korup di folder train dan test
# check_and_remove_corrupted_images(train_dir)
# check_and_remove_corrupted_images(test_dir)

In [8]:

# Parameter untuk ImageDataGenerator
batch_size = 32
img_height = 100
img_width = 100

# Membuat generator untuk data training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

# Membuat generator untuk data testing
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 603 images belonging to 25 classes.
Found 148 images belonging to 25 classes.


In [9]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    layers.MaxPooling2D(pool_size=(2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(train_generator.class_indices), activation='softmax')  # Jumlah kelas
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
!pip install imageio

import imageio
import os

def find_and_remove_corrupted_images(folder_path):
    """
    Finds and removes corrupted images within a given folder and its subfolders using imageio.

    Args:
        folder_path (str): The path to the folder containing the images.
    """
    for subdir, _, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(subdir, file)
            try:
                # Attempt to read the image using imageio
                _ = imageio.imread(file_path)
            except (imageio.core.format.CannotReadFrameError, ValueError) as e:
                # If imageio cannot read the image, it's likely corrupted
                print(f'Removing corrupted image: {file_path}')
                os.remove(file_path)

# Assuming 'train_dir' and 'test_dir' are defined as paths to your train and test folders
find_and_remove_corrupted_images(train_dir)
find_and_remove_corrupted_images(test_dir)

<ipython-input-10-d7ad070b30c5>:18: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  _ = imageio.imread(file_path)


In [11]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=20,  # Sesuaikan jumlah epoch
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size
)

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


18/18 ━━━━━━━━━━━━━━━━━━━━ 39s 1s/step - accuracy: 0.0428 - loss: 3.2271 - val_accuracy: 0.1094 - val_loss: 3.0771
Epoch 2/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.0625 - loss: 3.1561 - val_accuracy: 0.1500 - val_loss: 2.9565
Epoch 3/20


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


18/18 ━━━━━━━━━━━━━━━━━━━━ 32s 1s/step - accuracy: 0.1066 - loss: 3.0681 - val_accuracy: 0.1875 - val_loss: 2.8693
Epoch 4/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.1875 - loss: 2.8971 - val_accuracy: 0.1000 - val_loss: 2.9977
Epoch 5/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 39s 1s/step - accuracy: 0.1048 - loss: 2.8882 - val_accuracy: 0.1328 - val_loss: 2.8652
Epoch 6/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - accuracy: 0.1875 - loss: 2.7400 - val_accuracy: 0.2500 - val_loss: 2.5999
Epoch 7/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 38s 1s/step - accuracy: 0.1746 - loss: 2.7130 - val_accuracy: 0.2109 - val_loss: 2.5768
Epoch 8/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.1250 - loss: 2.9511 - val_accuracy: 0.1500 - val_loss: 2.5182
Epoch 9/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/step - accuracy: 0.2407 - loss: 2.5632 - val_accuracy: 0.2500 - val_loss: 2.4445
Epoch 10/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.1250 - loss: 2.8559 - val_accuracy: 0.2000 - val_loss: 2.5937
E

In [12]:
# Evaluasi model
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test accuracy: {test_acc}')

5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.3309 - loss: 2.1366
Test accuracy: 0.3378378450870514


In [14]:
# Melakukan prediksi
from google.colab import files
from tensorflow.keras.preprocessing import image
import numpy as np

uploaded = files.upload()

for fn in uploaded.keys():
  img_path = fn

img = image.load_img(img_path, target_size=(100, 100))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0

predictions = model.predict(img_array)

# Get the predicted class index
predicted_class_index = np.argmax(predictions)

# Get the class labels (assuming you have them in train_data.class_indices)
class_labels = list(train_generator.class_indices.keys())

# Print the predicted class label
predicted_class_label = class_labels[predicted_class_index]
print("Predicted Class:", predicted_class_label)

Saving anggur_test (4).jpg to anggur_test (4).jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
Predicted Class: anggur


In [16]:
# Melakukan prediksi
from google.colab import files
from tensorflow.keras.preprocessing import image
import numpy as np

uploaded = files.upload()

for fn in uploaded.keys():
  img_path = fn

img = image.load_img(img_path, target_size=(100, 100))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0

predictions = model.predict(img_array)

# Get the predicted class index
predicted_class_index = np.argmax(predictions)

# Get the class labels (assuming you have them in train_data.class_indices)
class_labels = list(train_generator.class_indices.keys())

# Print the predicted class label
predicted_class_label = class_labels[predicted_class_index]
print("Predicted Class:", predicted_class_label)

Saving bubur_test (3).jpg to bubur_test (3).jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Predicted Class: kerupuk


In [17]:
# Menyimpan model
model.save('model_makanan.h5')

In [18]:
!cp model_makanan.h5 /content/drive/MyDrive/cnnmakananbayi/